<a href="https://colab.research.google.com/github/Carlos-Pessin/LeNet5-for-image-classification/blob/main/LeNet5_DataOrganization_Kfold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import seaborn as sb
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score, confusion_matrix
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
from matplotlib import rcParams
import tensorflow as tf
import keras
from keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import UpSampling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import cv2
import skimage
import skimage.exposure
import skimage.transform


In [ ]:

#%% Import dos dados
canais = 3 	# cinza=1 / RGB=3
K.set_image_data_format('channels_last')


n_img = 600
height = 28   #tamanho exigido LeNet
width = 28
main_directory = '/content/drive/MyDrive/Pós/Redes Neurais/CNN/Lab DataOrganization/'

# extrai imagens diretamente de uma pasta. Subpastas dividem classes
dataset = image_dataset_from_directory(
          main_directory,
          labels = 'inferred',          # gera automaticamente os labels a partir da estrutura das subpastas
          label_mode = 'categorical',   # 'int' / 'binary
          #class_names = ['class_a', 'class_b'],
          color_mode = 'rgb',           # 'grayscale'
          batch_size = n_img,
          image_size = (height, width),
          shuffle = True,              # reordena as imagens aleatoriamente
          seed = 0,                    # para o shuffle
          validation_split = None,      # coloca a fracao de dados de treino para separacao automatica (0.8 por exemplo)
          # subset = None,
          interpolation = 'bilinear',    # metodo para resize da imagem
          follow_links = False         # para visitar subdiretorios
          )


for X,y in dataset:
  X = X.numpy()
  y = y.numpy()

plt.figure()
plt.imshow(X[0]/255)  #imshow le de 0 a 1

n_class = y.shape[1]


In [ ]:

# Construcao do modelo de rede
model = Sequential()

#%% Primeira Camada convolucional
n_filt1 = 16
model.add(Conv2D(filters=n_filt1,
                 kernel_size=(5,5),
                 strides=(1,1),
                 padding='same',  # mantem dimensao da entrada
                 activation='tanh',
                 input_shape=(28,28,3)))

model.add(AveragePooling2D(pool_size=(2,2), strides=(2,2)))

In [ ]:
#%% Segunda Camada convolucional
n_filt2 = 16
model.add(Conv2D(filters=n_filt2,
                 kernel_size=(5,5),
                 strides=(1,1),
                 padding='valid',   # nao faz padding, pode diminuir dimensao
                 activation='tanh',
                 input_shape=(14,14,16)))

model.add(AveragePooling2D(pool_size=(2,2), strides=(2,2)))

In [ ]:
#%% Flatten
model.add(Flatten())

In [ ]:
#%% Camadas neuronais
model.add(Dense(units=120, activation='tanh'))
model.add(Dense(units=84, activation='tanh'))
model.add(Dense(units=n_class, activation='softmax', name='predict'))

#%% Compila
opt = Adam(learning_rate=0.0001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


In [ ]:

confMatrix_KFold = []
results_KFold = []

confMatrix_tot = np.zeros((n_class, n_class))
result_med = np.zeros(2)

n_k = 4
kf = KFold(n_splits=n_k)

for train_index, test_index in kf.split(X):
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = y[train_index], y[test_index]

  # separa validacao e teste
  X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=0)   # divide o teste em teste e validacao

  # Treina
  early_stopping = EarlyStopping(monitor='val_accuracy', patience=250, restore_best_weights=True)   # define parada apos estagnacao de acuracia por 250 epocas, retoma melhores pesos
  history = model.fit(X_train, y_train, validation_data = (X_val, y_val), batch_size=32, epochs=500, verbose = 2, callbacks=[early_stopping])

  # Teste e avaliacao
  pred_test = np.argmax(model.predict(X_test), axis=-1)
  results = model.evaluate(X_test, y_test, batch_size=32)
  labels_test = np.argmax(y_test, axis=-1)
  confMatrix = confusion_matrix(labels_test, pred_test)

  results_KFold.append(results)
  confMatrix_KFold.append(confMatrix)

  results = np.array(results)
  result_med = result_med + results/n_k
  confMatrix = np.array(confMatrix)
  confMatrix_tot = confMatrix_tot + confMatrix

print(result_med)
print(confMatrix_tot)


Epoch 1/500
15/15 - 10s - loss: 0.6962 - accuracy: 0.5000 - val_loss: 0.7004 - val_accuracy: 0.4667 - 10s/epoch - 699ms/step
Epoch 2/500
15/15 - 0s - loss: 0.7007 - accuracy: 0.4911 - val_loss: 0.7446 - val_accuracy: 0.4400 - 88ms/epoch - 6ms/step
Epoch 3/500
15/15 - 0s - loss: 0.7029 - accuracy: 0.5000 - val_loss: 0.7383 - val_accuracy: 0.4400 - 106ms/epoch - 7ms/step
Epoch 4/500
15/15 - 0s - loss: 0.7285 - accuracy: 0.5044 - val_loss: 0.6966 - val_accuracy: 0.5600 - 89ms/epoch - 6ms/step
Epoch 5/500
15/15 - 0s - loss: 0.6964 - accuracy: 0.5156 - val_loss: 0.7269 - val_accuracy: 0.4400 - 104ms/epoch - 7ms/step
Epoch 6/500
15/15 - 0s - loss: 0.6878 - accuracy: 0.5489 - val_loss: 0.6837 - val_accuracy: 0.5600 - 87ms/epoch - 6ms/step
Epoch 7/500
15/15 - 0s - loss: 0.6942 - accuracy: 0.5222 - val_loss: 0.7272 - val_accuracy: 0.4400 - 90ms/epoch - 6ms/step
Epoch 8/500
15/15 - 0s - loss: 0.7091 - accuracy: 0.5044 - val_loss: 0.6831 - val_accuracy: 0.5600 - 93ms/epoch - 6ms/step
Epoch 9/500


In [ ]:
plt.figure()
plt.plot(history.history['accuracy'], label='train accuracy')
plt.plot(history.history['val_accuracy'], label='validation accuracy')
plt.title('Model Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epochs')
plt.legend('train', 'validation')
plt.tight_layout()
plt.show()

plt.figure()
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='validation loss')
plt.title('Model Loss')
plt.ylabel('loss')
plt.xlabel('epochs')
plt.legend('train', 'validation')
plt.tight_layout()
plt.show()